In [ ]:
from mpi4py import MPI
from dolfinx import mesh, fem
import numpy as np
from petsc4py.PETSc import ScalarType
import ufl
from dolfinx.fem.petsc import LinearProblem

msh = mesh.create_rectangle(
    comm=MPI.COMM_WORLD,
    points=((0.0,0.0),(1.0,1.0)),
    n=(16,16),
    cell_type=mesh.CellType.triangle
)

V = fem.functionspace(msh,("Lagrange",1))


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

def marker(x):
    return (
        np.isclose(x[0],0.0)
        | np.isclose(x[0],0.0)
        | np.isclose(x[0],1.0)
        | np.isclose(x[0],1.0)
    )

facets = mesh.locate_entities_boundary(msh,dim=(msh.topology.dim-1), marker=marker)
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)
bc = fem.dirichletbc(value=ScalarType(0),dofs=dofs,V=V)

# RHS

x = ufl.SpatialCoordinate(msh)
f = -2*(x[0]*(1-x[0]) + (1-x[1]))

a = ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx
L = ufl.inner(f,v)*ufl.dx

# Problem
problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options_prefix="test_poisson_",
    petsc_options={
        "ksp_type":"preonly",
        "pc_type":"lu",
        "ksp_error_if_not_converged":True
        }
)


ld: warning: duplicate -rpath '/Users/wilhelmfaerden/miniforge3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/Users/wilhelmfaerden/miniforge3/envs/fenicsx-env/lib' ignored
